In [1]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR, CONFIG_DIR
from src.external.hptr.src.data_modules.agent_centric import AgentCentricPreProcessing
from src.external.hptr.src.data_modules.ac_global import AgentCentricGlobal
from omegaconf import OmegaConf

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

2025-01-24 19:47:50.244 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\mimolm


g:\Work\DS\mimolm\notebooks


In [18]:
ARGO_SCENE_DIR = DATA_DIR / "raw" / "temp"
NUM_SCENARIOS = 1

all_scenario_files = sorted(ARGO_SCENE_DIR.rglob("*.parquet"))
scenario_file_list = (
    all_scenario_files[:NUM_SCENARIOS]
)

from av2.datasets.motion_forecasting import scenario_serialization
from av2.map.map_api import ArgoverseStaticMap

scenario_path = scenario_file_list[0]
scenario_id = scenario_path.stem.split("_")[-1]
static_map_path = (
    scenario_path.parents[0] / f"log_map_archive_{scenario_id}.json"
)

scenario = scenario_serialization.load_argoverse_scenario_parquet(scenario_path)
static_map = ArgoverseStaticMap.from_json(static_map_path)

In [24]:
from av2.datasets.motion_forecasting.viz.scenario_visualization import visualize_scenario

visualize_scenario(scenario, static_map, DATA_DIR / "processed" / "argoext.mp4")

In [ ]:
import h5py

with h5py.File(DATA_DIR / "interim" / "train.h5", "r", libver="latest", swmr=True) as hf:
    print(hf['11']['agent'].keys())

In [2]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2

data_module = DataH5av2(DATA_DIR / "interim")
data_module.setup(stage="fit")
dl = data_module.train_dataloader()

c:\Users\Harshavardhan Patil\.virtualenvs\mimolm-kv68Yv56\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
episode = None
prepreprocess = AgentCentricPreProcessing(time_step_current=49, 
                                          data_size=data_module.tensor_size_train, 
                                          n_target=6,
                                          n_other=50,
                                          n_map=512,
                                          mask_invalid=False)
preprocess = AgentCentricGlobal(time_step_current=49, 
                                data_size=data_module.tensor_size_train,
                                dropout_p_history=0.15, 
                                add_ohe=True,
                                pl_aggr=False,
                                pose_pe= {"agent": "xy_dir",
                                        "map": "mpa_pl"})

In [7]:
for batch in dl:
    batch = prepreprocess(batch=batch)
    batch = preprocess(batch=batch)
    episode = batch
    break

In [9]:
print(episode.keys())

dict_keys(['episode_idx', 'agent/valid', 'agent/pos', 'agent/vel', 'agent/spd', 'agent/acc', 'agent/yaw_bbox', 'agent/yaw_rate', 'agent/type', 'agent/cmd', 'agent/role', 'agent/size', 'agent/goal', 'agent/dest', 'map/valid', 'map/type', 'map/pos', 'map/dir', 'map/boundary', 'ref/idx', 'ref/idx_n', 'ref/pos', 'ref/rot', 'ref/type', 'ref/role', 'ac/target_valid', 'ac/target_pos', 'ac/target_vel', 'ac/target_spd', 'ac/target_acc', 'ac/target_yaw_bbox', 'ac/target_yaw_rate', 'ac/target_type', 'ac/target_role', 'ac/target_size', 'gt/valid', 'gt/spd', 'gt/pos', 'gt/vel', 'gt/yaw_bbox', 'gt/cmd', 'ac/other_valid', 'ac/other_spd', 'ac/other_acc', 'ac/other_yaw_rate', 'ac/other_pos', 'ac/other_vel', 'ac/other_yaw_bbox', 'ac/other_type', 'ac/other_role', 'ac/other_size', 'ac/map_valid', 'ac/map_type', 'ac/map_pos', 'ac/map_dir', 'input/target_type', 'input/target_valid', 'input/other_valid', 'input/map_valid', 'input/target_attr', 'input/other_attr', 'input/map_attr'])


In [ ]:
%run -i ../src/external/hptr/src/pack_h5_av2.py #input: data/raw/train || output: data/interi/train

2025-01-27 20:19:58.785 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\mimolm
100%|██████████| 1000/1000 [02:19<00:00,  7.19it/s]

n_pl_max: 762, n_agent_max: 155
n_agent_sim: 64, n_agent_no_sim: 91


In [ ]:
#python -m scenarionet.convert_argoverse2 -d data\interim --raw_data_path G:\Work\DS\mimolm\data\raw\train --overwrite

In [27]:
from scenarionet import read_dataset_summary, read_scenario

dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path=DATA_DIR / "interim" / "train")

### UniTraj Data Setup

In [4]:
from src.external.Unitraj.unitraj.datasets.base_dataset import BaseDataset
from omegaconf import OmegaConf

config = OmegaConf.load(CONFIG_DIR / "data.yml")
dataset = BaseDataset(config=config, is_validation=False)

Loading training data...
Using 8 processes to load data...
Processing data chunk 0...
G:/Work/DS/mimolm/data/train {'sd_av2_v2_00a0a3e0-1508-45f2-9cf5-e427e1446a33.pkl': 'train_0', 'sd_av2_v2_00a0adb0-6c55-4df6-88cd-6a524f4edb39.pkl': 'train_0', 'sd_av2_v2_00a2e7cc-f700-4a56-9b94-e312b66af424.pkl': 'train_0', 'sd_av2_v2_00a3a9a7-dd09-45fe-ac1e-24580e72ff14.pkl': 'train_0', 'sd_av2_v2_00a3d479-dc7e-4f39-8bc7-0c73c0e2f3a8.pkl': 'train_0', 'sd_av2_v2_00a4b70c-e650-4f2c-99f9-993f31f55f1d.pkl': 'train_0', 'sd_av2_v2_0a00c136-b7f1-4502-be51-55cd108005c7.pkl': 'train_0', 'sd_av2_v2_0a00dc81-63da-4a4f-aabd-24b1584edfe2.pkl': 'train_0', 'sd_av2_v2_0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca.pkl': 'train_0', 'sd_av2_v2_0a0a71c5-2e02-432c-ade6-dff22bc659de.pkl': 'train_0', 'sd_av2_v2_0a0b9607-f495-42c9-a52e-2d7156234597.pkl': 'train_0', 'sd_av2_v2_0a0bdc92-a48a-4c52-a082-9eeeb360a5bd.pkl': 'train_0', 'sd_av2_v2_0a0c3f36-2076-4033-b30a-a908c5c64ea4.pkl': 'train_1', 'sd_av2_v2_0a0c614c-763f-4880-b917-fd07

In [5]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=dataset, batch_size=3, num_workers=1, drop_last=False,
    collate_fn=dataset.collate_fn)

In [7]:
b = None
for batch in train_loader:
    print(batch.keys())
    b = batch
    break

dict_keys(['batch_size', 'input_dict', 'batch_sample_count'])


In [13]:
b['input_dict'].keys()

dict_keys(['center_gt_final_valid_idx', 'center_gt_trajs', 'center_gt_trajs_mask', 'center_gt_trajs_src', 'center_objects_id', 'center_objects_type', 'center_objects_world', 'dataset_name', 'kalman_difficulty', 'map_center', 'map_polylines', 'map_polylines_center', 'map_polylines_mask', 'obj_trajs', 'obj_trajs_future_mask', 'obj_trajs_future_state', 'obj_trajs_last_pos', 'obj_trajs_mask', 'obj_trajs_pos', 'scenario_id', 'track_index_to_predict', 'trajectory_type'])